# Converters

Converters are used to transform prompts before sending them to the target.

This can be useful for a variety of reasons, such as encoding the prompt in a different format, or adding additional information to the prompt. For example, you might want to convert a prompt to base64 before sending it to the target, or add a prefix to the prompt to indicate that it is a question.

## Simple Converter Example

Converters can be used to perform these types of transformations. Here is a simple program that uses Rot13Converter converter, RandomCapitalLettersConverter, and AsciiArtConverter.

In [ ]:
from pyrit.common import IN_MEMORY, initialize_pyrit
from pyrit.prompt_converter import (
    AsciiArtConverter,
    BinaryConverter,
    RandomCapitalLettersConverter,
    ROT13Converter,
)

initialize_pyrit(memory_db_type=IN_MEMORY)

prompt = "tell me how to cut down a tree"


print(await ROT13Converter().convert_tokens_async(prompt=prompt))  # type: ignore
print(await RandomCapitalLettersConverter(percentage=25.0).convert_tokens_async(prompt=prompt))  # type: ignore
print(await AsciiArtConverter().convert_tokens_async(prompt=prompt))  # type: ignore
print(await BinaryConverter().convert_tokens_async(prompt=prompt))  # type: ignore

text: gryy zr ubj gb phg qbja n gerr
text: TEll me how To cUt down a Tree
text:                                                                                                                                                                   
  o         8 8                    8                           o                           o         8                                     o                      
  8         8 8                    8                           8                           8         8                                     8                      
 o8P .oPYo. 8 8   ooYoYo. .oPYo.   8oPYo. .oPYo. o   o   o    o8P .oPYo.   .oPYo. o    o  o8P   .oPYo8 .oPYo. o   o   o odYo.   .oPYo.    o8P oPYo. .oPYo. .oPYo. 
  8  8oooo8 8 8   8' 8  8 8oooo8   8    8 8    8 Y. .P. .P     8  8    8   8    ' 8    8   8    8    8 8    8 Y. .P. .P 8' `8   .oooo8     8  8  `' 8oooo8 8oooo8 
  8  8.     8 8   8  8  8 8.       8    8 8    8 `b.d'b.d'     8  8    8   8    . 8    8   8    8    8 8 

In [ ]:
# Close connection
from pyrit.memory import CentralMemory

memory = CentralMemory.get_memory_instance()
memory.dispose_engine()